In [1]:
%pip install openai-agents nest_asyncio

Note: you may need to restart the kernel to use updated packages.


c:\Users\TempAccess\Documents\Dhruv\openai_agentic_sdk\memoryAgent\.venv\Scripts\python.exe: No module named pip


In [2]:
from openai import OpenAI

client = OpenAI()

In [3]:
import asyncio
from agents import Agent, Runner, set_tracing_disabled

set_tracing_disabled(True)

agent = Agent(
    name="Assistant",
    instructions="Reply very concisely.",
)
# Quick Test
result = await Runner.run(agent, "Tell me why it is important to evaluate AI agents.")
print(result.final_output)


Evaluating AI agents ensures they are reliable, safe, fair, and effective for their intended tasks. It helps identify errors, biases, and areas for improvement, increasing trust and accountability.


In [4]:
from dataclasses import dataclass, field
from typing import Any, Dict, List

@dataclass
class MemoryNote:
    text: str
    last_update_date: str
    keywords: List[str]


@dataclass
class TravelState:
    profile: Dict[str, Any] = field(default_factory=dict)

    # Long-term memory
    global_memory: Dict[str, Any] = field(default_factory=lambda: {"notes": []})

    # Short-term memory (staging for consolidation)
    session_memory: Dict[str, Any] = field(default_factory=lambda: {"notes": []})

    # Trip history (recent trips from DB)
    trip_history: Dict[str, Any] = field(default_factory=lambda: {"trips": []})

    # Rendered injection strings (computed per run)
    system_frontmatter: str = ""
    global_memories_md: str = ""
    session_memories_md: str = ""

    # Flag for triggering session injection after context trimming
    inject_session_memories_next_turn: bool = False


user_state = TravelState(
    profile={
        "global_customer_id": "crm_12345",
        "name": "John Doe",
        "age": "31",
        "home_city": "San Francisco",
        "currency" : "USD",
        "passport_expiry_date": "2029-06-12",
        "loyalty_status": {"airline": "United Gold", "hotel": "Marriott Titanium"},
        "loyalty_ids": {"marriott": "MR998877", "hilton": "HH445566", "hyatt": "HY112233"},
        "seat_preference": "aisle",
        "tone": "concise and friendly",
        "active_visas": ["Schengen", "US"],
        "insurance_coverage_profile": {
            "car_rental": "primary_cdw_included",
            "travel_medical": "covered",
        },
    },
    global_memory={
        "notes": [
            MemoryNote(
                text="For trips shorter than a week, user generally prefers not to check bags.",
                last_update_date="2025-04-05",
                keywords=["baggage", "short_trip"],
            ).__dict__,
            MemoryNote(
                text="User usually prefers aisle seats.",
                last_update_date="2024-06-25",
                keywords=["seat_preference"],
            ).__dict__,
            MemoryNote(
                text="User generally likes central, walkable city-center neighborhoods.",
                last_update_date="2024-02-11",
                keywords=["neighborhood"],
            ).__dict__,
            MemoryNote(
                text="User generally likes to compare options side-by-side",
                last_update_date="2023-02-17",
                keywords=["pricing"],
            ).__dict__,
            MemoryNote(
                text="User prefers high floors",
                last_update_date="2023-02-11",
                keywords=["room"],
            ).__dict__,
        ]
    },
    trip_history={
        "trips": [
            {
                # Core trip details
                "from_city": "Istanbul",
                "from_country": "Turkey",
                "to_city": "Paris",
                "to_country": "France",
                "check_in_date": "2025-05-01",
                "check_out_date": "2025-05-03",
                "trip_purpose": "leisure",  # leisure | business | family | etc.
                "party_size": 1,

                # Flight details
                "flight": {
                    "airline": "United",
                    "airline_status_at_booking": "United Gold",
                    "cabin_class": "economy_plus",
                    "seat_selected": "aisle",
                    "seat_location": "front",          # front | middle | back
                    "layovers": 1,
                    "baggage": {"checked_bags": 0, "carry_ons": 1},
                    "special_requests": ["vegetarian_meal"],  # optional
                },

                # Hotel details
                "hotel": {
                    "brand": "Hilton",
                    "property_name": "Hilton Paris Opera",
                    "neighborhood": "city_center",
                    "bed_type": "king",
                    "smoking": "non_smoking",
                    "high_floor": True,
                    "early_check_in": False,
                    "late_check_out": True,
                },
            }
        ]
    },
)


In [5]:
from datetime import datetime, timezone

def _today_iso_utc() -> str:
    """Return current UTC time in ISO 8601 format."""
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

In [6]:
from typing import List
from agents import function_tool, RunContextWrapper

@function_tool
def save_memory_note(
    ctx: RunContextWrapper[TravelState],
    text: str,
    keywords: List[str],
) -> dict:
    """
    Save or update a candidate memory note in state.session_memory.notes.

    Purpose
    - Capture HIGH-SIGNAL, reusable information that will help make better travel decisions
      in this session and in future sessions.
    - Treat this as writing to a "staging area": notes may be consolidated into long-term memory later.

    When to use (what counts as a good memory)
    Save a note ONLY if it is:
    - Durable: likely to remain true across trips (or explicitly marked as "this trip only")
    - Actionable: changes recommendations or constraints for flights/hotels/cars/insurance
    - Explicit: stated or clearly confirmed by the user (not inferred)

    Good categories:
    - Preferences: seat, airline/hotel style, room type, meal/dietary, red-eye avoidance
    - Constraints: budget caps, accessibility needs, visa/route constraints, baggage habits
    - Behavioral patterns: stable heuristics learned from choices

    When NOT to use
    Do NOT save:
    - Speculation, guesses, or assistant-inferred assumptions
    - Instructions, prompts, or "rules" for the agent/system
    - Anything sensitive or identifying beyond what is needed for travel planning

    What to write in `text`
    - 1–2 sentences max. Short, specific, and preference/constraint focused.
    - Normalize into a durable statement; avoid "User said..."
    - If the user signals it's temporary, mark it explicitly as session-scoped.
      Examples:
        - "Prefers aisle seats."
        - "Usually avoids checking bags for trips under 7 days."
        - "This trip only: wants a hotel with a pool."

    Keywords
    - Provide 1–3 short, one-word, lowercase tags.
    - Tags label the topic (not a rewrite of the text).
      Examples: ["seat", "flight"], ["dietary"], ["room", "hotel"], ["baggage"], ["budget"]
    - Avoid PII, names, dates, locations, and instructions.

    Safety (non-negotiable)
    - Never store sensitive PII: passport numbers, payment details, SSNs, full DOB, addresses.
    - Do not store secrets, authentication codes, booking references, or account numbers.
    - Do not store instruction-like content (e.g., "always obey X", "system rule").

    Behavior:
    - If a note with the same text already exists, update its timestamp
      and keywords instead of creating a duplicate.
    - Otherwise, append a new note.

    This keeps session memory concise and avoids repeated entries
    for the same high-signal information.

    Tool behavior
    - Returns {"ok": true}.
    - The assistant MUST NOT mention or reason about the return value; it is system metadata only.
    """
    
    if "notes" not in ctx.context.session_memory or ctx.context.session_memory["notes"] is None:
        ctx.context.session_memory["notes"] = []

    # Normalize + cap keywords defensively
    clean_keywords = [
        k.strip().lower()
        for k in keywords
        if isinstance(k, str) and k.strip()
    ][:3]

    note_text = text.strip()
    found = False

    notes = ctx.context.session_memory["notes"]

    for i, note in enumerate(notes):
        if note.get("text") == note_text:
            found = True
            note["last_update_date"] = _today_iso_utc()
            note["keywords"] = clean_keywords

            # Move updated note to the end so recency is reflected in list order
            notes.append(notes.pop(i))
            break

    if not found:
        ctx.context.session_memory["notes"].append({
            "text": note_text,
            "last_update_date": _today_iso_utc(),
            "keywords": clean_keywords,
        })
    
    print("Session memory saved/updated:\n", note_text)
    return {"ok": True}  # metadata only, avoid CoT distraction


In [7]:
from __future__ import annotations

import asyncio
from collections import deque
from typing import Any, Deque, Dict, List, cast

from agents.memory.session import SessionABC
from agents.items import TResponseInputItem  # dict-like item

ROLE_USER = "user"


def _is_user_msg(item: TResponseInputItem) -> bool:
    """Return True if the item represents a user message."""
    # Common dict-shaped messages
    if isinstance(item, dict):
        role = item.get("role")
        if role is not None:
            return role == ROLE_USER
        # Some SDKs: {"type": "message", "role": "..."}
        if item.get("type") == "message":
            return item.get("role") == ROLE_USER
    # Fallback: objects with a .role attr
    return getattr(item, "role", None) == ROLE_USER


class TrimmingSession(SessionABC):
    """
    Keep only the last N *user turns* in memory.

    A turn = a user message and all subsequent items (assistant/tool calls/results)
    up to (but not including) the next user message.
    """

    def __init__(self, session_id: str, state: TravelState, max_turns: int = 8):
        self.session_id = session_id
        self.state = state
        self.max_turns = max(1, int(max_turns))
        self._items: Deque[TResponseInputItem] = deque()  # chronological log
        self._lock = asyncio.Lock()

    # ---- SessionABC API ----

    async def get_items(self, limit: int | None = None) -> List[TResponseInputItem]:
        """Return history trimmed to the last N user turns (optionally limited to most-recent `limit` items)."""
        async with self._lock:
            trimmed = self._trim_to_last_turns(list(self._items))
            return trimmed[-limit:] if (limit is not None and limit >= 0) else trimmed

    async def add_items(self, items: List[TResponseInputItem]) -> None:
        """Append new items, then trim to last N user turns."""
        if not items:
            return
        async with self._lock:
            self._items.extend(items)
            original_len = len(self._items)
            trimmed = self._trim_to_last_turns(list(self._items))
            if len(trimmed) < original_len:
                # Flag for triggering session injection after context trimming
                self.state.inject_session_memories_next_turn = True
            self._items.clear()
            self._items.extend(trimmed)

    async def pop_item(self) -> TResponseInputItem | None:
        """Remove and return the most recent item (post-trim)."""
        async with self._lock:
            return self._items.pop() if self._items else None

    async def clear_session(self) -> None:
        """Remove all items for this session."""
        async with self._lock:
            self._items.clear()

    # ---- Helpers ----

    def _trim_to_last_turns(self, items: List[TResponseInputItem]) -> List[TResponseInputItem]:
        """
        Keep only the suffix containing the last `max_turns` user messages and everything after
        the earliest of those user messages.

        If there are fewer than `max_turns` user messages (or none), keep all items.
        """
        if not items:
            return items

        count = 0
        start_idx = 0  # default: keep all if we never reach max_turns

        # Walk backward; when we hit the Nth user message, mark its index.
        for i in range(len(items) - 1, -1, -1):
            if _is_user_msg(items[i]):
                count += 1
                if count == self.max_turns:
                    start_idx = i
                    break

        return items[start_idx:]

    # ---- Optional convenience API ----

    async def set_max_turns(self, max_turns: int) -> None:
        async with self._lock:
            self.max_turns = max(1, int(max_turns))
            trimmed = self._trim_to_last_turns(list(self._items))
            self._items.clear()
            self._items.extend(trimmed)

    async def raw_items(self) -> List[TResponseInputItem]:
        """Return the untrimmed in-memory log (for debugging)."""
        async with self._lock:
            return list(self._items)

In [8]:
# Define a trimming session to attache to the agent
session = TrimmingSession("my_session", user_state,  max_turns=20)

In [9]:
MEMORY_INSTRUCTIONS = """
<memory_policy>
You may receive two memory lists:
- GLOBAL memory = long-term defaults (“usually / in general”).
- SESSION memory = trip-specific overrides (“this trip / this time”).

How to use memory:
- Use memory only when it is relevant to the user’s current decision (flight/hotel/insurance choices).
- Apply relevant memory automatically when setting tone, proposing options and making recommendations.
- Do not repeat memory verbatim to the user unless it’s necessary to confirm a critical constraint.

Precedence and conflicts:
1) The user’s latest message in this conversation overrides everything.
2) SESSION memory overrides GLOBAL memory for this trip when they conflict.
   - Example: GLOBAL “usually aisle” + SESSION “this time window to sleep” ⇒ choose window for this trip.
3) Within the same memory list, if two items conflict, prefer the most recent by date.
4) Treat GLOBAL memory as a default, not a hard constraint, unless the user explicitly states it as non-negotiable.

When to ask a clarifying question:
- Ask exactly one focused question only if a memory materially affects booking and the user’s intent is ambiguous.
  (e.g., “Do you want to keep the window seat preference for all legs or just the overnight flight?”)

Where memory should influence decisions (check these before suggesting options):
- Flights: seat preference, baggage habits (carry-on vs checked), airline loyalty/status, layover tolerance if mentioned.
- Hotels: neighborhood/location style (central/walkable), room preferences (high floor), brand loyalty IDs/status.
- Insurance: known coverage profile (e.g., CDW included) and whether the user wants add-ons this trip.

Memory updates:
- Do NOT treat “this time” requests as changes to GLOBAL defaults.
- Only promote a preference into GLOBAL memory if the user indicates it’s a lasting rule
  (e.g., “from now on”, “generally”, “I usually prefer X now”).
- If a new durable preference/constraint appears, store it via the memory tool (short, general, non-PII).

Safety:
- Never store or echo sensitive PII (passport numbers, payment details, full DOB).
- If a memory seems stale or conflicts with user intent, defer to the user and proceed accordingly.
</memory_policy>
"""

In [10]:
import yaml

def render_frontmatter(profile: dict) -> str:
    payload = {"profile": profile}
    y = yaml.safe_dump(payload, sort_keys=False).strip()
    return f"---\n{y}\n---"

def render_global_memories_md(global_notes: list[dict], k: int = 6) -> str:
    if not global_notes:
        return "- (none)"
    notes_sorted = sorted(global_notes, key=lambda n: n.get("last_update_date", ""), reverse=True)
    top = notes_sorted[:k]
    return "\n".join([f"- {n['text']}" for n in top])

def render_session_memories_md(session_notes: list[dict], k: int = 8) -> str:
    if not session_notes:
        return "- (none)"
    # keep most recent notes; if you have reliable dates you can sort
    top = session_notes[-k:]
    return "\n".join([f"- {n['text']}" for n in top])

In [11]:
from agents import AgentHooks, Agent

class MemoryHooks(AgentHooks[TravelState]):
    def __init__(self, client: client):
        self.client = client

    async def on_start(self, ctx: RunContextWrapper[TravelState], agent: Agent) -> None:
        """
        Initialize agent memory context at the start of each run.

        This hook prepares:
        - system frontmatter (profile-based)
        - global memories (long-term)
        - session memories (short-term, conversation-specific)

        NOTE:
        Unlike the previous trim-gated behavior, session memories are now
        injected eagerly whenever they are available. This ensures that
        recent conversational context is consistently visible to the agent
        on every turn.
        """

        # Render system-level frontmatter from the user profile
        ctx.context.system_frontmatter = render_frontmatter(
            ctx.context.profile
        )

        # Render global (long-term) memories if present
        ctx.context.global_memories_md = render_global_memories_md(
            (ctx.context.global_memory or {}).get("notes", [])
        )

        # Fetch session-level (short-term) memory notes
        # These represent recent conversational context
        session_notes = (ctx.context.session_memory or {}).get("notes", [])

        # If session notes exist, always inject them into the context
        # This bypasses trim-based delayed injection and ensures
        # session memory is available on every turn
        if session_notes:
            ctx.context.session_memories_md = render_session_memories_md(
                session_notes
            )

        # Reset deferred-injection flag if it was set
        # This prevents duplicate or delayed session memory injection
        if ctx.context.inject_session_memories_next_turn:
            ctx.context.inject_session_memories_next_turn = False

    async def on_llm_start(
        self,
        context: RunContextWrapper,
        agent: Agent,
        system_prompt: Optional[str],
        input_items: list[TResponseInputItem],
    ) -> None:
        """
        Called immediately before the agent issues an LLM call.

        This hook is intentionally left empty but can be used
        for debugging, logging, or prompt inspection.
        """
        # Example for debugging:
        # print(f"\nSystem Prompt:\n{system_prompt}")
        pass

In [12]:
BASE_INSTRUCTIONS = f"""
You are a concise, reliable travel concierge. 
Help users plan and book flights, hotels, and car/travel insurance.\n\n

Guidelines:\n
- Collect key trip details and confirm understanding.\n
- Ask only one focused clarifying question at a time.\n
- Provide a few strong options with brief tradeoffs, then recommend one.\n
- Respect stable user preferences and constraints; avoid assumptions.\n
- Before booking, restate all details and get explicit approval.\n
- Never invent prices, availability, or policies—use tools or state uncertainty.\n
- Do not repeat sensitive PII; only request what is required.\n
- Track multi-step itineraries and unresolved decisions.\n\n

"""

In [13]:
async def instructions(ctx: RunContextWrapper[TravelState], agent: Agent) -> str:
    s = ctx.context

    session_block = ""

    # Ensure session memories are rendered if notes exist
    if not s.session_memories_md and (s.session_memory or {}).get("notes"):
        s.session_memories_md = render_session_memories_md(
            s.session_memory["notes"]
        )

    # Inject session memory eagerly whenever available
    if s.session_memories_md:
        session_block = (
            "\n\nSESSION memory (temporary; overrides GLOBAL when conflicting):\n"
            + s.session_memories_md
        )

        # Clear after injection to avoid duplication
        s.session_memories_md = ""
        s.inject_session_memories_next_turn = False

    return (
        BASE_INSTRUCTIONS
        + "\n\n<user_profile>\n" + (s.system_frontmatter or "") + "\n</user_profile>"
        + "\n\n<memories>\n"
        + "GLOBAL memory:\n" + (s.global_memories_md or "- (none)")
        + session_block
        + "\n</memories>"
        + "\n\n" + MEMORY_INSTRUCTIONS
    )


In [14]:
travel_concierge_agent = Agent(
    name="Travel Concierge",
    model="gpt-5.2",
    instructions=instructions,
    hooks=MemoryHooks(client),
    tools=[save_memory_note],
)

In [15]:
# Turn 1
r1 = await Runner.run(
    travel_concierge_agent,
    input="Book me a flight to Paris next month.",
    session=session,
    context=user_state,
)
print("Turn 1:", r1.final_output)

Turn 1: To book it, I need one detail first: what **departure city/airport** should I use (e.g., **SFO** or somewhere else)?

Once I have that, I’ll pull good options for **Paris (CDG vs ORY)** for next month—defaulting to **United/Star Alliance**, **aisle seat**, and **carry-on only** unless you tell me otherwise.


In [16]:
# Turn 2
r2 = await Runner.run(
    travel_concierge_agent,
    input="Do you know my preferences?",
    session=session,
    context=user_state,
)
print("\nTurn 2:", r2.final_output)


Turn 2: Yes—based on what I have for you:

- **Flights:** you usually prefer an **aisle seat** and, for trips **under a week**, you generally **avoid checking a bag**.
- **Hotels (if needed):** you tend to like **central, walkable neighborhoods** and **high floors**.
- **Loyalty:** **United Gold** and **Marriott Titanium**.

For this Paris flight: do you want me to **stick with aisle + carry-on only** for this trip?


In [17]:

# Turn 3 (should trigger save_memory_note)
r3 = await Runner.run(
    travel_concierge_agent,
    input="Remember that I am vegetarian.",
    session=session,
    context=user_state,
)
print("\nTurn 3:", r3.final_output)

Session memory saved/updated:
 Vegetarian meal preference.

Turn 3: Got it—I’ll make sure to request a **vegetarian meal** when we book.

What **departure airport/city** should I use for your Paris flight next month?


In [18]:
user_state.session_memory

{'notes': [{'text': 'Vegetarian meal preference.',
   'last_update_date': '2026-01-17T05:42:57+00:00',
   'keywords': ['dietary', 'meal']}]}

In [19]:

# Turn 4 (should trigger save_memory_note)
r4 = await Runner.run(
    travel_concierge_agent,
    input="This time, I like to have a window seat. I really want to sleep",
    session=session,
    context=user_state,
)
print("\nTurn 4:", r4.final_output)

Session memory saved/updated:
 This trip only: prefers a window seat to sleep (override usual aisle preference).

Turn 4: Understood—**window seat for this trip** so you can sleep.

What **departure airport/city** should I use for your Paris flight next month?


In [20]:
user_state.session_memory

{'notes': [{'text': 'Vegetarian meal preference.',
   'last_update_date': '2026-01-17T05:42:57+00:00',
   'keywords': ['dietary', 'meal']},
  {'text': 'This trip only: prefers a window seat to sleep (override usual aisle preference).',
   'last_update_date': '2026-01-17T05:43:01+00:00',
   'keywords': ['seat', 'flight']}]}

In [21]:
from __future__ import annotations

from typing import Any, Dict, List, Optional
import json


def consolidate_memory(state: TravelState, client, model: str = "gpt-5-mini") -> None:
    """
    Consolidate state.session_memory["notes"] into state.global_memory["notes"].

    - Merges duplicates / near-duplicates
    - Resolves conflicts by keeping most recent (last_update_date)
    - Clears session notes after consolidation
    - Mutates `state` in place
    """

    session_notes: List[Dict[str, Any]] = state.session_memory.get("notes", []) or []
    if not session_notes:
        return  # nothing to consolidate

    global_notes: List[Dict[str, Any]] = state.global_memory.get("notes", []) or []

    # Use json.dumps so the prompt contains valid JSON (not Python repr)
    global_json = json.dumps(global_notes, ensure_ascii=False)
    session_json = json.dumps(session_notes, ensure_ascii=False)

    consolidation_prompt = f"""
    You are consolidating travel memory notes into LONG-TERM (GLOBAL) memory.

    You will receive two JSON arrays:
    - GLOBAL_NOTES: existing long-term notes
    - SESSION_NOTES: new notes captured during this run

    GOAL
    Produce an updated GLOBAL_NOTES list by merging in SESSION_NOTES.

    RULES
    1) Keep only durable information (preferences, stable constraints, memberships/IDs, long-lived habits).
    2) Drop session-only / ephemeral notes. In particular, DO NOT add a note if it is clearly only for the current trip/session,
    e.g. contains phrases like "this time", "this trip", "for this booking", "right now", "today", "tonight", "tomorrow",
    or describes a one-off circumstance rather than a lasting preference/constraint.
    3) De-duplicate:
    - Remove exact duplicates.
    - Remove near-duplicates (same meaning). Keep a single best canonical version.
    4) Conflict resolution:
    - If two notes conflict, keep the one with the most recent last_update_date (YYYY-MM-DD).
    - If dates tie, prefer SESSION_NOTES over GLOBAL_NOTES.
    5) Note quality:
    - Keep each note short (1 sentence), specific, and durable.
    - Prefer canonical phrasing like: "Prefers aisle seats." / "Avoids red-eye flights." / "Has United Gold status."
    6) Do NOT invent new facts. Only use what appears in the input notes.

    OUTPUT FORMAT (STRICT)
    Return ONLY a valid JSON array.
    Each element MUST be an object with EXACTLY these keys:
    {{"text": string, "last_update_date": "YYYY-MM-DD", "keywords": [string]}}

    Do not include markdown, commentary, code fences, or extra keys.

    GLOBAL_NOTES (JSON):
    <GLOBAL_JSON>
    {global_json}
    </GLOBAL_JSON>

    SESSION_NOTES (JSON):
    <SESSION_JSON>
    {session_json}
    </SESSION_JSON>
    """.strip()


    resp = client.responses.create(
        model=model,
        input=consolidation_prompt,
    )

    consolidated_text = (resp.output_text or "").strip()

    # Parse safely (best-effort) and overwrite global notes
    try:
        consolidated_notes = json.loads(consolidated_text)
        if isinstance(consolidated_notes, list):
            state.global_memory["notes"] = consolidated_notes
        else:
            state.global_memory["notes"] = global_notes + session_notes
    except Exception:
        # If parsing fails, fall back to simple append
        state.global_memory["notes"] = global_notes + session_notes

    # Clear session memory after consolidation
    state.session_memory["notes"] = []


In [22]:
# Pre-consolidation session memories
user_state.session_memory

{'notes': [{'text': 'Vegetarian meal preference.',
   'last_update_date': '2026-01-17T05:42:57+00:00',
   'keywords': ['dietary', 'meal']},
  {'text': 'This trip only: prefers a window seat to sleep (override usual aisle preference).',
   'last_update_date': '2026-01-17T05:43:01+00:00',
   'keywords': ['seat', 'flight']}]}

In [23]:
# Pre-consolidation global memories
user_state.global_memory

{'notes': [{'text': 'For trips shorter than a week, user generally prefers not to check bags.',
   'last_update_date': '2025-04-05',
   'keywords': ['baggage', 'short_trip']},
  {'text': 'User usually prefers aisle seats.',
   'last_update_date': '2024-06-25',
   'keywords': ['seat_preference']},
  {'text': 'User generally likes central, walkable city-center neighborhoods.',
   'last_update_date': '2024-02-11',
   'keywords': ['neighborhood']},
  {'text': 'User generally likes to compare options side-by-side',
   'last_update_date': '2023-02-17',
   'keywords': ['pricing']},
  {'text': 'User prefers high floors',
   'last_update_date': '2023-02-11',
   'keywords': ['room']}]}

In [24]:
# Can be triggered when your app decides the session is “over” (explicit end, TTL, heartbeat)
consolidate_memory(user_state, client)

In [25]:
user_state.global_memory

{'notes': [{'text': 'Prefers not to check bags for trips shorter than a week.',
   'last_update_date': '2025-04-05',
   'keywords': ['baggage', 'short_trip']},
  {'text': 'Prefers aisle seats.',
   'last_update_date': '2024-06-25',
   'keywords': ['seat_preference']},
  {'text': 'Prefers central, walkable city-center neighborhoods.',
   'last_update_date': '2024-02-11',
   'keywords': ['neighborhood']},
  {'text': 'Likes to compare options side-by-side.',
   'last_update_date': '2023-02-17',
   'keywords': ['pricing']},
  {'text': 'Prefers high floors.',
   'last_update_date': '2023-02-11',
   'keywords': ['room']},
  {'text': 'Prefers vegetarian meals.',
   'last_update_date': '2026-01-17',
   'keywords': ['dietary', 'meal']}]}

In [26]:
user_state.session_memory

{'notes': []}